I started buy bringing all Citybikes dataframe to  complete this activity.

In [1]:
import requests
import json
import pandas as pd
import unidecode
import numpy as np
import pickle

In [2]:
# set city 
city_name = "boston"
print(city_name)

boston


In [3]:
# get response for all networks and normalize into dataframe
response = requests.request('GET',url="http://api.citybik.es/v2/networks")
data_networks = response.json()
df_networks = pd.json_normalize(data_networks['networks'])
print(df_networks)

                                       company  \
0                             [ЗАО «СитиБайк»]   
1               [Urban Infrastructure Partner]   
2                          [Comunicare S.r.l.]   
3                         [Cyclopolis Systems]   
4    [Groundwork, Slough Borough Council, ITS]   
..                                         ...   
607                             [PubliBike AG]   
608                             [PubliBike AG]   
609                             [PubliBike AG]   
610      [Citybike Normandie, INURBA Mobility]   
611         [The National Transport Authority]   

                                       href                         id  \
0              /v2/networks/velobike-moscow            velobike-moscow   
1              /v2/networks/baerum-bysykkel            baerum-bysykkel   
2             /v2/networks/bicincitta-siena           bicincitta-siena   
3          /v2/networks/cyclopolis-maroussi        cyclopolis-maroussi   
4            /v2/networks/cyc

In [4]:
# get list of cities, remove accents, and convert to lowercase
city_list = df_networks['location.city'].values
city_list = np.array([unidecode.unidecode(city).lower() for city in city_list])
print(city_list)

['moscow' 'baerum' 'siena' 'maroussi' 'slough' 'nafplio'
 'parco dei colli di bergamo' 'aigialeia' 'marathon' 'mantova'
 'nea smyrni' 'moschato-tavros' 'arxaia olympia' 'kiato' 'assemini'
 'rhodes' 'florina' 'tortoli' 'limnos' 'gaeta' 'borgarello'
 'pizzighettone e formigara' 'cork' 'limerick' 'bangkok' 'galway'
 'bielsko-biala' 'montecatini terme' 'bydgoszcz' 'london' 'aspen, co'
 'carrara' 'reykjavik' 'hamburg' 'sestri levante - lavagna' 'arezzo'
 'astana' 'paris' 'ascoli piceno' 'bergamo' 'girona' 'alba' 'milano'
 'biella' 'chivasso' 'zaragoza' 'cd de mexico' 'busto arsizio' 'antwerpen'
 'cuneo' 'malmo' 'las condes' 'madrid' 'didymoteicho' 'frosinone'
 'glyfada' 'igoumenitsa' 'lecce' 'ioannina' 'karditsa' 'manerba del garda'
 'kavala' 'kinouria' 'komotini' 'patra' 'marciana marina' 'rethymno'
 'samos' 'novara' 'nicosia' 'mani' 'irakleio' 'thessaloniki' 'athens'
 'padova' 'sao paulo' 'a coruna' 'ferrol - naron'
 'curtin university, perth, wa' 'melbourne, au' 'ferrara' 'montreal, qc'


In [10]:
# get list of endpoints for the specified city
hrefs = df_networks.loc[[True if 'boston' in city else False for city in city_list ],'href']
N_hrefs = hrefs.shape[0]
print(N_hrefs)


1


In [11]:
# set up a dataframe to store stations data
df = pd.DataFrame()

In [13]:
df

""


In [14]:
# go through each endpoint (there will likely only be one)
for href in hrefs.values:
    print("http://api.citybik.es" + href)

http://api.citybik.es/v2/networks/blue-bikes


In [15]:
# get the stations data for that endpoint
response = requests.request('GET',url="http://api.citybik.es" + href + "?fields=stations")
data = response.json()
df1 = pd.json_normalize(data)

print(df1)

                                    network.stations
0  [{'empty_slots': 10, 'extra': {'ebikes': 0, 'h...


In [36]:
# extract the stations only and add to the dataframe
stations = data['network']['stations']
df_new = pd.json_normalize(stations)
df = pd.concat([df, df_new])
print(df)

                       fsq_id  \
0    55d32677498e12fc85b37dcb   
1    5de119c170624675cb935aac   
2    38510484832d45009bcc436e   
3    d6e4b1f061e14d58121332ae   
4    7f0e7017232d417934b62ff1   
..                        ...   
450                       NaN   
451                       NaN   
452                       NaN   
453                       NaN   
454                       NaN   

                                            categories  \
0    [{'id': 19014, 'name': 'Hotel', 'icon': {'pref...   
1    [{'id': 19053, 'name': 'Limo Service', 'icon':...   
2    [{'id': 19036, 'name': 'Airport Service', 'ico...   
3    [{'id': 19010, 'name': 'Bed and Breakfast', 'i...   
4    [{'id': 19053, 'name': 'Limo Service', 'icon':...   
..                                                 ...   
450                                                NaN   
451                                                NaN   
452                                                NaN   
453                    

In [37]:
df.head(10)

,fsq_id,categories,chains,distance,geocodes,link,location,name,related_places,timezone,...,timestamp,extra.ebikes,extra.has_ebikes,extra.last_updated,extra.payment,extra.payment-terminal,extra.renting,extra.returning,extra.slots,extra.uid
0,55d32677498e12fc85b37dcb,"[{'id': 19014, 'name': 'Hotel', 'icon': {'pref...",[{'id': '9518916e-93aa-4468-b2e0-dbd8e151e1a5'...,605.0,"{'drop_off': {'latitude': 42.389921, 'longitud...",/v3/places/55d32677498e12fc85b37dcb,"{'address': '100 Boardman St', 'census_block':...",Hilton Garden Inn Boston Logan Airport,{'children': []},America/New_York,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5de119c170624675cb935aac,"[{'id': 19053, 'name': 'Limo Service', 'icon':...",[],241.0,"{'drop_off': {'latitude': 42.387993, 'longitud...",/v3/places/5de119c170624675cb935aac,"{'address': '1 Blackinton St', 'census_block':...",Logan Limo Transport East Boston,{},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,38510484832d45009bcc436e,"[{'id': 19036, 'name': 'Airport Service', 'ico...",[],579.0,"{'drop_off': {'latitude': 42.391402, 'longitud...",/v3/places/38510484832d45009bcc436e,"{'address': '232 Orient Ave', 'census_block': ...",Airport Impact Relief East Boston,{},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,d6e4b1f061e14d58121332ae,"[{'id': 19010, 'name': 'Bed and Breakfast', 'i...",[],621.0,"{'main': {'latitude': 42.392361, 'longitude': ...",/v3/places/d6e4b1f061e14d58121332ae,"{'address': '14 Crestway Rd', 'census_block': ...",East Boston B&B,{},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7f0e7017232d417934b62ff1,"[{'id': 19053, 'name': 'Limo Service', 'icon':...",[],890.0,"{'main': {'latitude': 42.385937, 'longitude': ...",/v3/places/7f0e7017232d417934b62ff1,"{'address': '175 William F McClellan Hwy', 'ce...",Boston Luxury Limo,{},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1dbb65270fd54c15901a957d,"[{'id': 19049, 'name': 'Taxi Stand', 'icon': {...",[],890.0,"{'main': {'latitude': 42.385937, 'longitude': ...",/v3/places/1dbb65270fd54c15901a957d,"{'address': '175 William F McClellan Hwy', 'ce...",East Boston Taxi Cab,{},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3bba75ef69ac44c9b35983cc,"[{'id': 19049, 'name': 'Taxi Stand', 'icon': {...",[],897.0,"{'main': {'latitude': 42.385937, 'longitude': ...",/v3/places/3bba75ef69ac44c9b35983cc,"{'address': '175 William F McClellan Hwy', 'ce...",Boston Taxi,{},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,f691a0cb62eb4c14a0af60a1,"[{'id': 19053, 'name': 'Limo Service', 'icon':...",[],900.0,"{'main': {'latitude': 42.385992, 'longitude': ...",/v3/places/f691a0cb62eb4c14a0af60a1,"{'address': '175 William F McClellan Hwy', 'ce...",Elite Coach of Boston,{},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0019d4afff374ceaade1ac50,"[{'id': 19053, 'name': 'Limo Service', 'icon':...",[],902.0,"{'main': {'latitude': 42.385882, 'longitude': ...",/v3/places/0019d4afff374ceaade1ac50,"{'address': '175 William F McClellan Hwy', 'ce...",Boston Charter Bus,{},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,758ceda4cdad4eba40aa46c9,"[{'id': 19053, 'name': 'Limo Service', 'icon':...",[],967.0,"{'drop_off': {'latitude': 42.386385, 'longitud...",/v3/places/758ceda4cdad4eba40aa46c9,"{'address': '175 William F McClellan Hwy', 'ad...",Boston Professional Limousine,{},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df.count()

empty_slots               455
free_bikes                455
id                        455
latitude                  455
longitude                 455
name                      455
timestamp                 455
extra.ebikes              455
extra.has_ebikes          455
extra.last_updated        455
extra.payment             455
extra.payment-terminal    455
extra.renting             455
extra.returning           455
extra.slots               455
extra.uid                 455
dtype: int64

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [38]:

##To request for foursquare   with radius 1000m for all the bike stations in Boston i used the following codes.
import requests

url = "https://api.foursquare.com/v3/places/search?query=boston&ll=42.38678%2C-71.00609&radius=1000&categories=19000"

headers = {
    "accept": "application/json",
    "Authorization": '<API KEY>'
}
response = requests.get(url, headers=headers)

print(response.text)

{"results":[{"fsq_id":"55d32677498e12fc85b37dcb","categories":[{"id":19014,"name":"Hotel","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/travel/hotel_","suffix":".png"}}],"chains":[{"id":"9518916e-93aa-4468-b2e0-dbd8e151e1a5","name":"Hilton Garden Inn"}],"distance":605,"geocodes":{"drop_off":{"latitude":42.389921,"longitude":-71.012299},"front_door":{"latitude":42.389975,"longitude":-71.011983},"main":{"latitude":42.389975,"longitude":-71.011983},"roof":{"latitude":42.38978,"longitude":-71.012277}},"link":"/v3/places/55d32677498e12fc85b37dcb","location":{"address":"100 Boardman St","census_block":"250250510002009","country":"US","cross_street":"","dma":"Boston","formatted_address":"100 Boardman St, Boston, MA 02128","locality":"Boston","postcode":"02128","region":"MA"},"name":"Hilton Garden Inn Boston Logan Airport","related_places":{"children":[]},"timezone":"America/New_York"},{"fsq_id":"5de119c170624675cb935aac","categories":[{"id":19053,"name":"Limo Service","icon":{"pref

In [39]:
response = response.json()

# Convert JSON to DataFrame
foursquare_df = pd.DataFrame(response['results'])


In [40]:
foursquare_df

,fsq_id,categories,chains,distance,geocodes,link,location,name,related_places,timezone
0,55d32677498e12fc85b37dcb,"[{'id': 19014, 'name': 'Hotel', 'icon': {'pref...",[{'id': '9518916e-93aa-4468-b2e0-dbd8e151e1a5'...,605,"{'drop_off': {'latitude': 42.389921, 'longitud...",/v3/places/55d32677498e12fc85b37dcb,"{'address': '100 Boardman St', 'census_block':...",Hilton Garden Inn Boston Logan Airport,{'children': []},America/New_York
1,5de119c170624675cb935aac,"[{'id': 19053, 'name': 'Limo Service', 'icon':...",[],241,"{'drop_off': {'latitude': 42.387993, 'longitud...",/v3/places/5de119c170624675cb935aac,"{'address': '1 Blackinton St', 'census_block':...",Logan Limo Transport East Boston,{},NaN
2,38510484832d45009bcc436e,"[{'id': 19036, 'name': 'Airport Service', 'ico...",[],579,"{'drop_off': {'latitude': 42.391402, 'longitud...",/v3/places/38510484832d45009bcc436e,"{'address': '232 Orient Ave', 'census_block': ...",Airport Impact Relief East Boston,{},NaN
3,d6e4b1f061e14d58121332ae,"[{'id': 19010, 'name': 'Bed and Breakfast', 'i...",[],621,"{'main': {'latitude': 42.392361, 'longitude': ...",/v3/places/d6e4b1f061e14d58121332ae,"{'address': '14 Crestway Rd', 'census_block': ...",East Boston B&B,{},NaN
4,7f0e7017232d417934b62ff1,"[{'id': 19053, 'name': 'Limo Service', 'icon':...",[],890,"{'main': {'latitude': 42.385937, 'longitude': ...",/v3/places/7f0e7017232d417934b62ff1,"{'address': '175 William F McClellan Hwy', 'ce...",Boston Luxury Limo,{},NaN
5,1dbb65270fd54c15901a957d,"[{'id': 19049, 'name': 'Taxi Stand', 'icon': {...",[],890,"{'main': {'latitude': 42.385937, 'longitude': ...",/v3/places/1dbb65270fd54c15901a957d,"{'address': '175 William F McClellan Hwy', 'ce...",East Boston Taxi Cab,{},NaN
6,3bba75ef69ac44c9b35983cc,"[{'id': 19049, 'name': 'Taxi Stand', 'icon': {...",[],897,"{'main': {'latitude': 42.385937, 'longitude': ...",/v3/places/3bba75ef69ac44c9b35983cc,"{'address': '175 William F McClellan Hwy', 'ce...",Boston Taxi,{},NaN
7,f691a0cb62eb4c14a0af60a1,"[{'id': 19053, 'name': 'Limo Service', 'icon':...",[],900,"{'main': {'latitude': 42.385992, 'longitude': ...",/v3/places/f691a0cb62eb4c14a0af60a1,"{'address': '175 William F McClellan Hwy', 'ce...",Elite Coach of Boston,{},NaN
8,0019d4afff374ceaade1ac50,"[{'id': 19053, 'name': 'Limo Service', 'icon':...",[],902,"{'main': {'latitude': 42.385882, 'longitude': ...",/v3/places/0019d4afff374ceaade1ac50,"{'address': '175 William F McClellan Hwy', 'ce...",Boston Charter Bus,{},NaN
9,758ceda4cdad4eba40aa46c9,"[{'id': 19053, 'name': 'Limo Service', 'icon':...",[],967,"{'drop_off': {'latitude': 42.386385, 'longitud...",/v3/places/758ceda4cdad4eba40aa46c9,"{'address': '175 William F McClellan Hwy', 'ad...",Boston Professional Limousine,{},NaN


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [41]:
#Foursquare parsing
import pandas as pd
import requests

#  Boston bike stations 'df' with station data
foursquare_api_key = <API KEY>

URL = "https://api.foursquare.com/v3/places/search"

header_dict = {
    "Accept": "application/json",
    "Authorization": foursquare_api_key
}

a = []
b = []
c = []
d = []
e = []
f = []

for index, row in df.iterrows():
    # Extract station coordinates
    lat = row['latitude']
    lng = row['longitude']
    location = f"{lat},{lng}"
    # Set parameters for Foursquare API request
    params = {
        "query": "restaurant",
        "ll": location,
        "radius": "1000"
    }

    response = requests.get(URL, params=params, headers=header_dict)
    res_json = response.json()
    try:
        for i in res_json["results"]:
            a.append(i.get("name", None))  # Append name to 'a' list
            if "address" in i.get("location", {}):
                b.append(i["location"]["address"])  # Append address to 'b' list
            else:
                b.append(None)
            if "categories" in i:
                c.append(i["categories"][0].get("id", None))  # Append category ID to 'c' list
                d.append(i["categories"][0].get("name", None))  # Append category name to 'd' list
            else:
                c.append(None)
                d.append(None)
            e.append(i["location"].get("postcode", None))  # Append postcode to 'e' list
            f.append(i.get("distance", None))  # Append distance to 'f' list
    except:
        pass

foursquare_df = pd.DataFrame({"name": a, "address": b, "category_id": c, "category_name": d, "postcode": e, "distance": f})

# Print or do further processing with the dataframe 'df'
print(foursquare_df)

                            name             address  category_id  \
0             Donna's Restaurant     980 Saratoga St        13068   
1          New Saigon Restaurant       Bennington St        13379   
2                    Little Asia   991 Bennington St        13072   
3     Orient Heights Food Market  1016 Bennington St        17069   
4              El Paisa Butchery  1010 Bennington St        13039   
...                          ...                 ...          ...   
4491     Picco Pizza & Ice Cream      513 Tremont St        13236   
4492         Blackbird Doughnuts      492 Tremont St        13043   
4493                     Dunkin'  1138 Washington St        13001   
4494            Kava Neo-Taverna     315 Shawmut Ave        13003   
4495           South End Buttery     314 Shawmut Ave        13002   

              category_name postcode  distance  
0       American Restaurant    02128        93  
1     Vietnamese Restaurant    02128       117  
2          Asian Restaur

Put your parsed results into a DataFrame

In [42]:
foursquare_df

,name,address,category_id,category_name,postcode,distance
0,Donna's Restaurant,980 Saratoga St,13068,American Restaurant,02128,93
1,New Saigon Restaurant,Bennington St,13379,Vietnamese Restaurant,02128,117
2,Little Asia,991 Bennington St,13072,Asian Restaurant,02128,92
3,Orient Heights Food Market,1016 Bennington St,17069,Grocery Store,02128,25
4,El Paisa Butchery,1010 Bennington St,13039,Deli,02128,51
...,...,...,...,...,...,...
4491,Picco Pizza & Ice Cream,513 Tremont St,13236,Italian Restaurant,02116,253
4492,Blackbird Doughnuts,492 Tremont St,13043,Donut Shop,02116,256
4493,Dunkin',1138 Washington St,13001,Bagel Shop,02118,149
4494,Kava Neo-Taverna,315 Shawmut Ave,13003,Bar,02118,300


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [24]:
# Yelp for boston bikes stations around 1000m.
import requests

url = "https://api.yelp.com/v3/businesses/search?location=boston&latitude=42.38678&longitude=-71.00609&term=%27bike%20station%27&radius=1000&sort_by=rating&limit=50"

headers = {
    "accept": "application/json",
    "Authorization": '<API KEY>'
}

response = requests.get(url, headers=headers)

print(response.text)

{"businesses": [], "total": 0, "region": {"center": {"longitude": -71.00609, "latitude": 42.38678}}}


In [32]:
##for yelp
import os
import requests

api_key = '<API KEY>'

URL = "https://api.yelp.com/v3/businesses/search" 

params = {
    "term": "restaurant",
    "location": "Boston",
    "open_now": "true",
    "radius": "1000"
}

header_dict = {
    "Accept": "application/JSON",
    "Authorization": "Bearer " + '<API KEY>'
}


response = requests.get(URL, params=params, headers=header_dict)
print(response.text)

{"businesses": [{"id": "r28bxKFNTRKaXYcemS8KNw", "alias": "off-the-common-boston", "name": "Off the Common", "image_url": "https://s3-media1.fl.yelpcdn.com/bphoto/CAneqM4TBD4Mv0-G3z7Xew/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/off-the-common-boston?adjust_creative=Z_BjCdW_S1d8DrkkIpWTUA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=Z_BjCdW_S1d8DrkkIpWTUA", "review_count": 42, "categories": [{"alias": "newamerican", "title": "American (New)"}], "rating": 3.5, "coordinates": {"latitude": 42.3510474708316, "longitude": -71.0696800944176}, "transactions": ["delivery"], "price": "$$", "location": {"address1": "50 Park Plz", "address2": "", "address3": "Boston Park Plaza", "city": "Boston", "zip_code": "02116", "country": "US", "state": "MA", "display_address": ["50 Park Plz", "Boston Park Plaza", "Boston, MA 02116"]}, "phone": "+16174572330", "display_phone": "(617) 457-2330", "distance": 378.9062517016187}, {"id": "9mClygXHgT32dBxwBvGLww", "alias

In [33]:
yelp_json=response.json()

yelp_json

{'businesses': [{'id': 'r28bxKFNTRKaXYcemS8KNw',
   'alias': 'off-the-common-boston',
   'name': 'Off the Common',
   'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/CAneqM4TBD4Mv0-G3z7Xew/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/off-the-common-boston?adjust_creative=Z_BjCdW_S1d8DrkkIpWTUA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=Z_BjCdW_S1d8DrkkIpWTUA',
   'review_count': 42,
   'categories': [{'alias': 'newamerican', 'title': 'American (New)'}],
   'rating': 3.5,
   'coordinates': {'latitude': 42.3510474708316,
    'longitude': -71.0696800944176},
   'transactions': ['delivery'],
   'price': '$$',
   'location': {'address1': '50 Park Plz',
    'address2': '',
    'address3': 'Boston Park Plaza',
    'city': 'Boston',
    'zip_code': '02116',
    'country': 'US',
    'state': 'MA',
    'display_address': ['50 Park Plz',
     'Boston Park Plaza',
     'Boston, MA 02116']},
   'phone': '+16174572330',
   'display_phone': '(617

In [34]:
yelp_json["businesses"]

[{'id': 'r28bxKFNTRKaXYcemS8KNw',
  'alias': 'off-the-common-boston',
  'name': 'Off the Common',
  'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/CAneqM4TBD4Mv0-G3z7Xew/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/off-the-common-boston?adjust_creative=Z_BjCdW_S1d8DrkkIpWTUA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=Z_BjCdW_S1d8DrkkIpWTUA',
  'review_count': 42,
  'categories': [{'alias': 'newamerican', 'title': 'American (New)'}],
  'rating': 3.5,
  'coordinates': {'latitude': 42.3510474708316,
   'longitude': -71.0696800944176},
  'transactions': ['delivery'],
  'price': '$$',
  'location': {'address1': '50 Park Plz',
   'address2': '',
   'address3': 'Boston Park Plaza',
   'city': 'Boston',
   'zip_code': '02116',
   'country': 'US',
   'state': 'MA',
   'display_address': ['50 Park Plz',
    'Boston Park Plaza',
    'Boston, MA 02116']},
  'phone': '+16174572330',
  'display_phone': '(617) 457-2330',
  'distance': 378.90625170

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [43]:
# for yelp
import pandas as pd
import requests
import os

# Assuming you have a dataframe called 'austinbikes_df' with station data
yelp_key = '<API KEY>'

URL = "https://api.yelp.com/v3/businesses/search"

header_dict = {
    "Accept": "application/JSON",
    "Authorization": "Bearer " + '<API KEY>'
}
a = []
b = []
c = []
d = []
e = []
f = []
g = []
h = []
j = []

for index, row in df.iterrows():
    # Extract station coordinates
    lat = row['latitude']
    lng = row['longitude']
    location = f"{lat},{lng}"

    # Set parameters for Yelp API request
    params = {
        "term": "restaurant",
        "location": location,
        "radius": "1000"
    }

    response = requests.get(URL, params=params, headers=header_dict)
    res_json = response.json()

    try:
        for i in res_json["businesses"]:
            a.append(i.get("name", None))  # Append name to 'a' list
            if "location" in i and "address1" in i["location"]:
                b.append(i["location"]["address1"])  # Append address to 'b' list
                j.append(i["location"]["zip_code"])
            else:
                b.append(None)
                j.append(None)
            c.append(i.get("distance", None))  # Append transactions to 'c' list
            if "categories" in i and isinstance(i["categories"], list) and len(i["categories"]) > 0:
                d.append(i["categories"][0].get("title", None))  # Append category name to 'd' list
            else:
                d.append(None)
            e.append(i.get("review_count", None))  # Append review count to 'e' list
            f.append(i.get("rating", None))  # Append rating to 'f' list
            g.append(i["coordinates"].get("longitude", None))  # Append longitude to 'g' list
            h.append(i["coordinates"].get("latitude", None))  # Append latitude to 'h' list
    except:
        pass

yelp_df = pd.DataFrame({"name": a, "address": b, "distance": c, "category_name": d, "review_count": e, "rating": f, "longitude": g, "latitude": h, "postcode": j})

# Print or do further processing with the dataframe 'yelp_df'
print(yelp_df)


print(response.text)

                                             name             address  \
0                  Angela's Cafe - Orient Heights  1012 Bennington St   
1     Luna's Restaurant Italian and Latin Cuisine     945 Saratoga St   
2                                      New Saigon   985 Bennington St   
3                              Donna's Restaurant     980 Saratoga St   
4                                  Renegade's Pub  1004 Bennington St   
...                                           ...                 ...   
8788                                      Cósmica      40 Berkeley St   
8789                          Burro Bar South End  1357 Washington St   
8790                                  Bar Mezzana    360 Harrison Ave   
8791                                     Ali Baba   145 E Berkeley St   
8792                                  The Beehive      541 Tremont St   

        distance           category_name  review_count  rating  longitude  \
0      33.503191                 Mexican      

Put your parsed results into a DataFrame

In [44]:
yelp_df

,name,address,distance,category_name,review_count,rating,longitude,latitude,postcode
0,Angela's Cafe - Orient Heights,1012 Bennington St,33.503191,Mexican,141,4.5,-71.006310,42.387100,02128
1,Luna's Restaurant Italian and Latin Cuisine,945 Saratoga St,245.389220,Italian,124,4.5,-71.009160,42.386590,02128
2,New Saigon,985 Bennington St,107.990329,Vietnamese,205,4.5,-71.007323,42.386337,02128
3,Donna's Restaurant,980 Saratoga St,85.462869,American (Traditional),182,4.0,-71.007180,42.387020,02128
4,Renegade's Pub,1004 Bennington St,48.676351,Pubs,125,4.0,-71.006800,42.386890,02128
...,...,...,...,...,...,...,...,...,...
8788,Cósmica,40 Berkeley St,354.431304,Mexican,136,3.5,-71.070870,42.345940,02116
8789,Burro Bar South End,1357 Washington St,285.282081,Mexican,228,4.0,-71.069311,42.341577,02118
8790,Bar Mezzana,360 Harrison Ave,343.896926,Italian,427,4.0,-71.063579,42.344576,02118
8791,Ali Baba,145 E Berkeley St,233.143882,Turkish,245,4.0,-71.064816,42.343613,02118


In [ ]:
df 

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

To compare the completeness of Foursquare and Yelp API results, I consider the following:

1.Number of Records:  The Yelp API with a higher number of records has more complete data. this include the following(business, id,alias, name,.coordinates, review counts, ratingsl,ocation, phone number, distance.etc) whic centers around my Point Of Interest(POI), While Foursquare has limited  records such as (fsqid, geocodes, category name etc) which are restricted columns.

2.Missing Values: both API that provided fewer missing values indicates better completeness.

3.Required Fields: Yelp API returned a better and essential fields or attributes  such as review_count and rating which are not available in Foursquare API and all these are required for better analysis
4.Consistency: Both API responses are consistent across different requests but Yelp API is mo robust in completness. 
5.Data Validity: Data returned by each  both APIs Look consistent without outliers but Foursquare API gave fewer output.

In [ ]:
import requests

# Make API requests and get the results
api_url1 = "https://api.foursquare.com/v3/places/search"
api_url2 = "https://api.yelp.com/v3/businesses/search"

response1 = requests.get(api_url1)
response2 = requests.get(api_url2)

# Extract the data from the responses (assuming the data is in JSON format)
data1 = response1.json()
data2 = response2.json()

# Compare the number of records
num_records1 = len(data1["name","address", "category_id", "category_name", "postcode", "distance"])
num_records2 = len(data2["name", "address", "distance", "category_name", "review_count", "rating", "longitude", "latitude", "postcode"])

print("API 1 Number of Records:", num_records1)
print("API 2 Number of Records:", num_records2)

# Compare the missing values
missing_values1 = data1['missing_values']
missing_values2 = data2['missing_values']

print("API 1 Missing Values:", missing_values1)
print("API 2 Missing Values:", missing_values2)


Get the top 10 restaurants according to their rating

In [45]:
 #From Yelp_df
top_10_restaurants = yelp_df.sort_values(by='rating', ascending=False).head(10)
top_10_restaurants

,name,address,distance,category_name,review_count,rating,longitude,latitude,postcode
7005,La Casa Del Pandebono,271 Meridian St,987.080091,Bakeries,49,5.0,-71.038930,42.376310,02128
4955,Sp Cafe,70 Everett Ave,373.997793,Cafes,2,5.0,-71.039160,42.393150,02150
4947,antojitos hondurenos,139 Arlington St,612.321821,Honduran,4,5.0,-71.038220,42.393550,02150
4934,Sabor Salvadoreño,700 Broadway,275.680217,Latin American,8,5.0,-71.028481,42.396229,02150
853,Dynasty,14 Hudson St,682.519374,Cantonese,14,5.0,-71.059908,42.350613,02111
7552,Via Cannuccia,1739 Dorchester Ave,1203.123593,Breakfast & Brunch,31,5.0,-71.062992,42.290939,02124
4926,antojitos hondurenos,139 Arlington St,373.746145,Honduran,4,5.0,-71.038220,42.393550,02150
7554,Fusion Kitchen,855 Washington St,25.559068,Trinidadian,1,5.0,-71.071410,42.281873,02124
4919,Sp Cafe,70 Everett Ave,347.226208,Cafes,2,5.0,-71.039160,42.393150,02150
4918,AbondiZ Caribbean Cuisine,103 A Broadway,306.130044,Haitian,6,5.0,-71.041457,42.388453,02150
